Bu çalışmada, e-posta mesajlarının spam olup olmadığını belirlemek amacıyla bir sınıflandırma modeli geliştirilecektir. Metin verisi temel ön işleme adımlarından geçirilip, ardından Logistic Regression modeli ile eğitilecek ve test edilecektir.

In [13]:
# Veri seti içe aktarılıyor ve yalnızca etiket (spam/ham) ile mesaj içeriği sütunları seçiliyor.
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding='ISO-8859-1',usecols=[0,1])
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
# Rename Columns

df = df.rename(columns={'v1': 'label', 'v2': 'text'})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Bu adımda, veri setindeki sütun adlarını daha anlamlı hale getirmek için yeniden adlandırdım. ‘v1’ sütunu ‘label’, ‘v2’ sütunu ise ‘text’ olarak değiştirildi.

In [15]:
def review_df(df):
  print("Shape")
  print(df.shape)
  print("-"*100)
  print("Info")
  print(df.info())
  print("-"*100)
  print("Null")
  print(df.isnull().sum())
  print("-"*100)
  print("Head")
  print(df.head())
  print("-"*100)
  print("Tail")
  print(df.tail())
  print("-"*100)

review_df(df)

Shape
(5572, 2)
----------------------------------------------------------------------------------------------------
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   object
 1   text    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
----------------------------------------------------------------------------------------------------
Null
label    0
text     0
dtype: int64
----------------------------------------------------------------------------------------------------
Head
  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to us

Bu adımda, veri setinin genel yapısını incelemek için özel bir kontrol fonksiyonu oluşturdum. Satır/sütun sayısı, veri tipleri, eksik değerler ve ilk/son birkaç örnek gözlemlendi.

## Preprocessing
Modeli eğitmeden önce, metin verisi üzerinde bazı ön işleme adımları uygulamam gerekiyor. Bu adımlar, verinin daha temiz, tutarlı ve makine öğrenmesi algoritmaları için uygun hale getirilmesini sağlar. Bu süreçte gereksiz karakterler temizlenecek, metinler standartlaştırılacak ve analiz için daha anlamlı hale getirilecektir.

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocessing_pipeline(df, new_column, source_column):

    print("1. Lowercase")
    df[new_column] = df[source_column].str.lower()

    print("2. Remove punctuation and digits")
    df[new_column] = df[new_column].str.replace('[^\w\s]', '', regex=True)
    df[new_column] = df[new_column].str.replace('\d+', '', regex=True)

    print("3. Remove stopwords")
    stop_words = set(stopwords.words('english'))
    df[new_column] = df[new_column].apply(lambda x: " ".join(
        word for word in x.split() if word not in stop_words))

    print("4. Strip whitespaces")
    df[new_column] = df[new_column].str.strip()

    print("5. Tokenization")
    df[new_column] = df[new_column].apply(lambda x: word_tokenize(x))

    print("Preprocessing complete!")
    return df


Bu adımda, metin verisini temizlemek ve modele uygun hale getirmek için bir ön işleme pipeline'ı oluşturdum. Bu süreçte metinler küçültüldü, noktalama işaretleri ve sayılar kaldırıldı, anlamsız (stopword) kelimeler temizlendi ve her mesaj kelimelere ayrılarak tokenize edildi.

In [17]:
df = preprocessing_pipeline(df, new_column='normalized_text', source_column='text')

1. Lowercase
2. Remove punctuation and digits
3. Remove stopwords
4. Strip whitespaces
5. Tokenization
Preprocessing complete!


In [18]:
df['normalized_text'][0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [19]:
df['text'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [20]:
#Label Mapping

label_mapping = {'ham': 0, 'spam': 1}
df['label'] = df['label'].map(label_mapping)

In [21]:
df['label'].head(15)

,label
0,0
1,0
2,1
3,0
4,0
5,1
6,0
7,0
8,1
9,1


Bu adımda, ‘ham’ ve ‘spam’ şeklindeki metinsel etiketleri sayısal değerlere dönüştürdüm. ‘ham’ = 0, ‘spam’ = 1 olacak şekilde etiketleme yapıldı. Bu, modelin sınıfları daha iyi öğrenebilmesi için gerekli

## Modelling

Ön işleme adımları tamamlandıktan sonra, veriyi kullanarak bir sınıflandırma modeli oluşturuyorum. Bu bölümde eğitim ve test veri kümeleri ayrılacak, ardından Logistic Regression algoritması ile model eğitilip değerlendirme metrikleri üzerinden performansı analiz edilecektir

In [22]:
# Vectorize Text for train to machine

from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()

X = df['normalized_text'].apply(lambda x: ' '.join(x))
y = df['label']


X_tfidf = vectorizer.fit_transform(X)


Bu adımda, metin verilerini sayısal forma dönüştürmek için TF-IDF vektörleştirme yöntemi uyguladım. Her mesaj, kelime ağırlıklarına göre temsil edilerek makine öğrenmesi algoritmalarının anlayabileceği bir yapıya getirildi.

In [23]:
# Train Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


print("Model Score: ", model.score(X_test, y_test))
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model Score:  0.947085201793722
Confusion Matrix:
[[961   4]
 [ 55  95]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       0.96      0.63      0.76       150

    accuracy                           0.95      1115
   macro avg       0.95      0.81      0.87      1115
weighted avg       0.95      0.95      0.94      1115



Bu adımda, veriyi eğitim ve test olmak üzere ikiye ayırdım. Ardından Logistic Regression modelini eğittim ve test verisi üzerinde performansını değerlendirdim.

Modelin genel doğruluk skoru %94,7 olarak elde edildi. Ham (0) sınıfı oldukça yüksek başarıyla tahmin edilirken, spam (1) sınıfı için recall değeri %63 ile daha düşük kaldı. Bu da modelin bazı spam e-postaları kaçırdığını gösteriyor. Precision ise her iki sınıf için oldukça yüksek, bu da modelin tahmin ettiği spam maillerin çoğunun gerçekten spam olduğunu gösteriyor. Genel olarak model dengeli bir performans sergilemiş olsa da, spam sınıfındaki recall değerini artırmak için veri dengesizliği veya model iyileştirme çalışmaları yapılabilir.

In [26]:
test_text = "There is a free prize if you share your account informations with we."

prediction = model.predict(vectorizer.transform([test_text]))[0]
print("Tahmin:", "Spam" if prediction == 1 else "Ham")

Tahmin: Spam


Eğittiğim modeli kullanarak yeni bir e-posta örneğinin spam olup olmadığını test ettim. Verilen örnek, TF-IDF ile dönüştürüldükten sonra modele verildi ve model bu mesajın spam olup olmadığını tahmin etti.
Sonuç olarak spam olduğunu tahmin etti ve bu doğruydu.

Bu çalışmada, e-posta mesajlarının spam olup olmadığını belirlemek için temel bir metin sınıflandırma pipeline’ı kurdum. Veriler ön işleme adımlarından geçirildi, TF-IDF ile vektörleştirildi ve Logistic Regression modeli ile eğitildi. Model, %94 üzeri doğruluk sağladı ve örnek bir mesajı başarılı şekilde spam olarak tahmin etti.